In [1]:
from PIL import Image 
from IPython.display import display 
import random
import json
import os

In [3]:
dir_path = os.getcwd() + "\\trait-layers"
items = {}
category_list = []
arr = []
for category in os.listdir(dir_path):
    category_path = os.path.join(dir_path,category)
    category_list.append(category)
    for item in os.listdir(category_path):
        arr.append(item)
    items[category] = arr.copy()
    arr.clear()
print(items)

{'animal': ['cat@3x.png', 'penguin@3x.png', 'puppy@3x.png', 'rabbit@3x.png', 'turtle@3x.png'], 'backgrounds': ['blue.jpg', 'orange.jpg', 'purple.jpg', 'red.jpg', 'yellow.jpg'], 'bottom': ['blackoveralls@3x.png', 'denimpants@3x.png', 'greenoveralls@3x.png', 'koreanpants@3x.png', 'scoutpants@3x.png'], 'hair': ['glasses@3x.png', 'koreanhat@3x.png', 'ribbon@3x.png', 'sunglassees@3x.png'], 'hand': ['coke@3x.png', 'letter@3x.png', 'pencil@3x.png', 'phone@3x.png'], 'shoes': ['blackshoes@3x.png', 'greenshoes@3x.png', 'pinkshoes@3x.png', 'rainboots@3x.png'], 'top': ['anipaltop@3x.png', 'bluetop@3x.png', 'koreangown_1@3x.png', 'scouttop@3x.png', 'startop@3x.png', 'stripetop@3x.png']}


In [4]:
## Generate Traits

TOTAL_IMAGES = 30 # Number of random unique images we want to generate

all_images = [] 

def getWeights(items):
    weightsFor4 = [30, 50, 15, 5]
    weightsFor5 = [30, 40, 15, 5, 10]
    weightsFor6 = [25, 35, 15, 3, 10, 12]
    if len(items) == 4:
        return weightsFor4
    elif len(items) == 5:
        return weightsFor5
    else:
        return weightsFor6

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} #

    # For each trait category, select a random trait based on the weightings 
    for i in range(len(category_list)):
        c = category_list[i]
        new_image [c] = random.choices(items[c], getWeights(items[c]))[0]
    
    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


In [5]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

Are all images unique? True


In [6]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [7]:
print(all_images)

[{'animal': 'cat@3x.png', 'backgrounds': 'orange.jpg', 'bottom': 'greenoveralls@3x.png', 'hair': 'koreanhat@3x.png', 'hand': 'letter@3x.png', 'shoes': 'blackshoes@3x.png', 'top': 'anipaltop@3x.png', 'tokenId': 0}, {'animal': 'puppy@3x.png', 'backgrounds': 'orange.jpg', 'bottom': 'greenoveralls@3x.png', 'hair': 'glasses@3x.png', 'hand': 'letter@3x.png', 'shoes': 'blackshoes@3x.png', 'top': 'bluetop@3x.png', 'tokenId': 1}, {'animal': 'puppy@3x.png', 'backgrounds': 'purple.jpg', 'bottom': 'blackoveralls@3x.png', 'hair': 'koreanhat@3x.png', 'hand': 'phone@3x.png', 'shoes': 'greenshoes@3x.png', 'top': 'bluetop@3x.png', 'tokenId': 2}, {'animal': 'puppy@3x.png', 'backgrounds': 'blue.jpg', 'bottom': 'denimpants@3x.png', 'hair': 'glasses@3x.png', 'hand': 'letter@3x.png', 'shoes': 'greenshoes@3x.png', 'top': 'bluetop@3x.png', 'tokenId': 3}, {'animal': 'puppy@3x.png', 'backgrounds': 'blue.jpg', 'bottom': 'koreanpants@3x.png', 'hair': 'glasses@3x.png', 'hand': 'letter@3x.png', 'shoes': 'greenshoes

In [ ]:
# Get Trait Counts

background_count = {}
for item in background:
    background_count[item] = 0
    
circle_count = {}
for item in circle:
    circle_count[item] = 0

square_count = {}
for item in square:
    square_count[item] = 0

for image in all_images:
    background_count[image["Background"]] += 1
    circle_count[image["Circle"]] += 1
    square_count[image["Square"]] += 1
    
print(background_count)
print(circle_count)
print(square_count)

In [ ]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [ ]:
  
    
#### Generate Images    
for item in all_images:

    im1 = Image.open(f'./trait-layers/backgrounds/{background_files[item["Background"]]}.jpg').convert('RGBA')
    im2 = Image.open(f'./trait-layers/circles/{circle_files[item["Circle"]]}.png').convert('RGBA')
    im3 = Image.open(f'./trait-layers/squares/{square_files[item["Square"]]}.png').convert('RGBA')

    #Create each composite
    com1 = Image.alpha_composite(im1, im2)
    com2 = Image.alpha_composite(com1, im3)

    #Convert to RGB
    rgb_im = com2.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)
    
    
    

In [ ]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_BASE_URI = "ADD_IMAGES_BASE_URI_HERE"
PROJECT_NAME = "ADD_PROJECT_NAME_HERE"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image": IMAGES_BASE_URI + str(token_id) + '.png',
        "tokenId": token_id,
        "name": PROJECT_NAME + ' ' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Circle", i["Circle"]))
    token["attributes"].append(getAttribute("Square", i["Square"]))

    with open('./metadata/' + str(token_id), 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()